## Assignment: Working with Dependency Graphs (Parses)

The objective of the assignment is to learn how to work with dependency graphs by defining functions.

Read [spaCy documentation on dependency parser](https://spacy.io/api/dependencyparser) to learn provided methods.

Define functions to:

- extract a path of dependency relations from the ROOT to a token
    - input is a sentence, you parse it and get a Doc object of spaCy
    - for each token the path will be a list of dependency relations, where first element is ROOT
- extract subtree of a dependents given a token
    - input is a sentence, you parse it and get a Doc object of spaCy
    - for each token in Doc objects you extract a subtree of its dependents as a list (ordered w.r.t. sentence order)
- check if a given list of tokens (segment of a sentence) forms a subtree
    - you parse a sentence and get a Doc object of spaCy
    - providing as an input ordered list of words from a sentence, you output True/False based on the sequence forming a subtree or not
- identify head of a span, given its tokens
    - input is a sequence of words (not necessarily a sentence)
    - output is the head of the span (single word)
- extract sentence subject, direct object and indirect object spans
    - input is a sentence, you parse it and get a Doc object of spaCy
    - output is lists of words that form a span (not a single word) for subject, direct object, and indirect object (if present of course, otherwise empty)
        - dict of lists, is better

In [1]:
import spacy
spacy_nlp = spacy.load('en_core_web_sm')

# sentence = "The objective of the assignment is to learn how to work with dependency graphs by defining functions."
# sentence = "Pierre Vinken, 61 years old, will join the board as a nonexecutive director Nov. 29."
# sentence = "bright red apples on the tree"
# sentence = "Autonomous cars shift insurance liability toward manufacturers"
# sentence = "The teacher gave the class some homework."
sentence = "Sue passed Ann the ball."
# sentence = "I saw the man."
# sentence = "The quick brown fox jumps over the lazy dog."

#display the dep graph
doc1 = spacy_nlp(sentence)
spacy.displacy.render(doc1, style='dep')

## First point

- extract a path of dependency relations from the ROOT to a token
    - input is a sentence, you parse it and get a Doc object of spaCy
    - for each token the path will be a list of dependency relations, where first element is ROOT

In this point, my idea was to split the problem into two functions:
- the first function __pathDep(token)__ return the path from the root to the token as a normal graph visiting using a FIFO queue. The __input__ is a *spaCy token* and the __output__ is a *list* with inside the tuple *(dependecy_of_the_node, text_of_the_node)*, I added the text to have a more clear output.

- the second function __printPathDep(sent)__ prints all the paths of the tokens inside the sentence. As __input__ a *string* representing the sentence

In [2]:
# visit as a normal graph visiting
def pathDep(token):
    S = []
    path = []
    S.append(token)
    while len(S) > 0:
        node = S.pop(0)
        path.append((node.dep_, node.text))
        if node.dep_ != "ROOT":
            S.append(node.head)
    return list(reversed(path))

def printPathDep(sent):
    doc = spacy_nlp(sent)
    path = []
    for token in doc:
        path = pathDep(token)
        print(path)

printPathDep(sentence)

[('ROOT', 'passed'), ('nsubj', 'Sue')]
[('ROOT', 'passed')]
[('ROOT', 'passed'), ('dative', 'Ann')]
[('ROOT', 'passed'), ('dobj', 'ball'), ('det', 'the')]
[('ROOT', 'passed'), ('dobj', 'ball')]
[('ROOT', 'passed'), ('punct', '.')]


## Second point

- extract subtree of a dependents given a token
    - input is a sentence, you parse it and get a Doc object of spaCy
    - for each token in Doc objects you extract a subtree of its dependents as a list (ordered w.r.t. sentence order)

Also in this point I decided to split the problem in two function, so that they can be used in the other points:
- the first function __subTreeDep(token)__ wants a sapCy token as input an return the list representing the subtree of the token (iterating over the token.subtree as in spaCy docs)

- the second function __printSubTreeDep(sent)__ receives as input a sentence and print all the subtree of the tokens inside, by iterating over all the tokens. It could be possible to output a list of list containing all the path for each token, but I think that the first function gives the flexibility to do that.

In [3]:
def subTreeDep(token):
    tree = []
    for sub in token.subtree:
        tree.append(sub)
    return tree

def printSubTreeDep(sent):
    doc = spacy_nlp(sent)
    for token in doc:
        path = subTreeDep(token)
        print(path)

printSubTreeDep(sentence)

[Sue]
[Sue, passed, Ann, the, ball, .]
[Ann]
[the]
[the, ball]
[.]


## Third point

- check if a given list of tokens (segment of a sentence) forms a subtree
    - you parse a sentence and get a Doc object of spaCy
    - providing as an input ordered list of words from a sentence, you output True/False based on the sequence forming a subtree or not

In the third point, the function __chekSubTree(sent, tokenList)__ get as input the *string* of the sentence to parse, and the *list* of tokens to check if they are contained in a subtree of the sentence. This function reuse *subTreeDep* to retrieve the subtree of each token to then compare it with the passed token list. It returns a *Boolean* as __output__.

For a quick test I create a list of word from a sentence and then convert it into a sorted list of tokens with the function __createTokenList(string)__

In [4]:
# Pre-create a list of word from a sentence
def createTokenList(string):
    tok = spacy_nlp(string)
    lt = []
    for t in tok:
        lt.append(str(t))
    return sorted(lt)

tmp1 = "the ball"
tmp2 = "Ann the"
listTokens = createTokenList(tmp1)
wrongTokens = createTokenList(tmp2)
print(listTokens)
print(wrongTokens)

['ball', 'the']
['Ann', 'the']


In [5]:
def checkSubTree(sent, tokenlist):
    doc = spacy_nlp(sent)
    for token in doc:
        path = subTreeDep(token)
        pathString = sorted([str(t) for t in path])
        if pathString == tokenlist:
            return True
    return False

print("Check if {} forms a subtree: {}".format(listTokens, checkSubTree(sentence, listTokens)))
print("Check if {} forms a subtree: {}".format(wrongTokens, checkSubTree(sentence, wrongTokens)))

Check if ['ball', 'the'] forms a subtree: True
Check if ['Ann', 'the'] forms a subtree: False


## Fourth point

- identify head of a span, given its tokens
    - input is a sequence of words (not necessarily a sentence)
    - output is the head of the span (single word)

Here my doubts were if the input of the function can be a standard *string* or a sliced spaCy document for this reason, I develop:

- __headSpan(span)__ receive as __input__ a slice of a spaCy document and return the root by accessing directly to the span propriety

- __headSpanString(span)__ receive as __input__ a *string* of list of word and then convert it do a spaCy document for the computing of the root

In [6]:
doc = spacy_nlp("Autonomous cars shift insurance liability toward manufacturers.")
def headSpan(span):
    return span.root.text

span = doc[3:6]
print("Span: {}".format(span))
print("Head of span: {}".format(headSpan(span)))

Span: insurance liability toward
Head of span: liability


In [7]:
def headSpanString(span):
    doc = spacy_nlp(span)
    return doc[0:len(doc)].root.text

print("Head of span: {}".format(headSpanString("insurance liability toward")))

Head of span: liability


## Fifth point

- extract sentence subject, direct object and indirect object spans
    - input is a sentence, you parse it and get a Doc object of spaCy
    - output is lists of words that form a span (not a single word) for subject, direct object, and indirect object (if present of course, otherwise empty)
        - dict of lists, is better

This function __extractDep(sent)__ receive in input a *string* of the sentence, convert it as a spaCy doc and returns a *dictionary* containing all the span object for the *subject*, *indirect object* and *direct object* by iterating in its subtree. After iterating the subtree, I decided to convert it into a span object and save it in the dictionary.
During some test and looking at the dependency graph, the best way to "detect" the indirect object was to use the word *dative* instead of *iobj*. I have not found any documentation about that on spaCy *iobj*, but only for *dative* in [spacy doc](https://spacy.io/models/en). For example, for the sentence "Sue passed Ann the ball.", the indirect object Ann is detected by the dative keyword.

In [8]:
def extractDep(sent):
    doc = spacy_nlp(sent)
    dic = {}
    dic["nsubj"] = None
    dic["iobj"] = None
    dic["dobj"] = None
    s = []
    i = []
    do = []
    
    for token in doc:
        
        if token.dep_ == "nsubj":
            for d in token.subtree:
                    s.append(d.i)
            span = doc[s[0]:s[len(s)-1]+1]
            dic["nsubj"] = span
            
        if token.dep_ == "dative":
            for d in token.subtree:
                    i.append(d.i)
            span = doc[i[0]:i[len(i)-1]+1]
            dic["iobj"] = span
            
        if token.dep_ == "dobj":
            for d in token.subtree:
                    do.append(d.i)
            span = doc[do[0]:do[len(do)-1]+1]
            dic["dobj"] = span
            
    return dic

print(extractDep(sentence))

{'nsubj': Sue, 'iobj': Ann, 'dobj': the ball}
